# Two-Stage Optimization

This notebook demonstrates a **two-stage optimization** approach:

1. **Stage 1: Sizing** - Optimize component sizes using downsampled data
2. **Stage 2: Dispatch** - Re-optimize with fixed sizes at full resolution

This technique is useful for:
- **Large models** where full optimization is computationally expensive
- **Faster development** iterations
- **Investment planning** where sizing decisions can be approximated first

We'll compare this approach to a combined (single-stage) optimization.

## Setup

In [1]:
import pathlib
import timeit

import pandas as pd

import flixopt as fx

fx.CONFIG.notebook()

## Load Data

In [2]:
# Load time series data
data_path = pathlib.Path('..') / '..' / 'examples' / 'resources' / 'Zeitreihen2020.csv'
data_import = pd.read_csv(data_path, index_col=0).sort_index()

# Use first 500 timesteps for this example
filtered_data = data_import[:500]
filtered_data.index = pd.to_datetime(filtered_data.index)
timesteps = filtered_data.index

print(f'Time range: {timesteps[0]} to {timesteps[-1]}')
print(f'Number of timesteps: {len(timesteps)}')

Time range: 2020-01-01 00:00:00 to 2020-01-06 04:45:00
Number of timesteps: 500


In [3]:
# Extract time series
electricity_demand = filtered_data['P_Netz/MW'].to_numpy()
heat_demand = filtered_data['Q_Netz/MW'].to_numpy()
electricity_price = filtered_data['Strompr.€/MWh'].to_numpy()
gas_price = filtered_data['Gaspr.€/MWh'].to_numpy()

## Build FlowSystem with Investment Options

We create a FlowSystem with `InvestParameters` for component sizing:

In [4]:
flow_system = fx.FlowSystem(timesteps)

flow_system.add_elements(
    # Buses
    fx.Bus('Strom', carrier='electricity'),
    fx.Bus(
        'Fernwärme', carrier='heat', imbalance_penalty_per_flow_hour=1e5
    ),  # High penalty for unmet demand - allows feasibility while strongly discouraging imbalance
    fx.Bus('Gas', carrier='gas'),
    fx.Bus('Kohle', carrier='fuel'),
    # Effects
    fx.Effect('costs', '€', 'Kosten', is_standard=True, is_objective=True),
    fx.Effect('CO2', 'kg', 'CO2_e-Emissionen'),
    fx.Effect('PE', 'kWh_PE', 'Primärenergie'),
    # Gas Boiler with investment optimization
    fx.linear_converters.Boiler(
        'Kessel',
        thermal_efficiency=0.85,
        thermal_flow=fx.Flow(label='Q_th', bus='Fernwärme'),
        fuel_flow=fx.Flow(
            label='Q_fu',
            bus='Gas',
            size=fx.InvestParameters(
                effects_of_investment_per_size={'costs': 1_000},
                minimum_size=10,
                maximum_size=600,
            ),
            relative_minimum=0.2,
            previous_flow_rate=20,
            status_parameters=fx.StatusParameters(effects_per_startup=300),
        ),
    ),
    # CHP with investment optimization
    fx.linear_converters.CHP(
        'BHKW2',
        thermal_efficiency=0.58,
        electrical_efficiency=0.22,
        status_parameters=fx.StatusParameters(effects_per_startup=1_000, min_uptime=10, min_downtime=10),
        electrical_flow=fx.Flow('P_el', bus='Strom'),
        thermal_flow=fx.Flow('Q_th', bus='Fernwärme'),
        fuel_flow=fx.Flow(
            'Q_fu',
            bus='Kohle',
            size=fx.InvestParameters(
                effects_of_investment_per_size={'costs': 3_000},
                minimum_size=10,
                maximum_size=500,
            ),
            relative_minimum=0.3,
            previous_flow_rate=100,
        ),
    ),
    # Storage with investment optimization
    fx.Storage(
        'Speicher',
        capacity_in_flow_hours=fx.InvestParameters(
            effects_of_investment_per_size={'costs': 60},
            minimum_size=10,
            maximum_size=1000,
        ),
        initial_charge_state='equals_final',
        eta_charge=1,
        eta_discharge=1,
        relative_loss_per_hour=0.001,
        prevent_simultaneous_charge_and_discharge=True,
        charging=fx.Flow('Q_th_load', size=200, bus='Fernwärme'),
        discharging=fx.Flow('Q_th_unload', size=200, bus='Fernwärme'),
    ),
    # Sinks and Sources
    fx.Sink('Wärmelast', inputs=[fx.Flow('Q_th_Last', bus='Fernwärme', size=1, fixed_relative_profile=heat_demand)]),
    fx.Source(
        'Gastarif',
        outputs=[fx.Flow('Q_Gas', bus='Gas', size=1000, effects_per_flow_hour={'costs': gas_price, 'CO2': 0.3})],
    ),
    fx.Source(
        'Kohletarif',
        outputs=[fx.Flow('Q_Kohle', bus='Kohle', size=1000, effects_per_flow_hour={'costs': 4.6, 'CO2': 0.3})],
    ),
    fx.Source(
        'Einspeisung',
        outputs=[
            fx.Flow(
                'P_el', bus='Strom', size=1000, effects_per_flow_hour={'costs': electricity_price + 0.5, 'CO2': 0.3}
            )
        ],
    ),
    fx.Sink('Stromlast', inputs=[fx.Flow('P_el_Last', bus='Strom', size=1, fixed_relative_profile=electricity_demand)]),
    fx.Source(
        'Stromtarif',
        outputs=[
            fx.Flow('P_el', bus='Strom', size=1000, effects_per_flow_hour={'costs': electricity_price, 'CO2': 0.3})
        ],
    ),
)

2025-12-08 15:15:32.585 INFO     │ Registered new Bus: Strom
2025-12-08 15:15:32.587 INFO     │ Registered new Bus: Fernwärme
2025-12-08 15:15:32.588 INFO     │ Registered new Bus: Gas
2025-12-08 15:15:32.589 INFO     │ Registered new Bus: Kohle
2025-12-08 15:15:32.590 INFO     │ Registered new Effect: costs
2025-12-08 15:15:32.590 INFO     │ Registered new Effect: costs
2025-12-08 15:15:32.595 INFO     │ Registered new Effect: CO2
2025-12-08 15:15:32.611 INFO     │ Registered new Effect: CO2
2025-12-08 15:15:32.616 INFO     │ Registered new Effect: PE
2025-12-08 15:15:32.624 INFO     │ Registered new Effect: PE
2025-12-08 15:15:32.627 INFO     │ Registered new Boiler: Kessel
2025-12-08 15:15:32.631 INFO     │ Registered new CHP: BHKW2
2025-12-08 15:15:32.633 INFO     │ Registered new Storage: Speicher
2025-12-08 15:15:32.636 INFO     │ Registered new Sink: Wärmelast
2025-12-08 15:15:32.637 INFO     │ Registered new Source: Gastarif
2025-12-08 15:15:32.638 INFO     │ Registered new Sou

## Two-Stage Optimization

### Stage 1: Sizing with Downsampled Data

We use `transform.resample('2h')` to reduce the problem size by half. This speeds up the investment optimization significantly:

In [5]:
solver = fx.solvers.HighsSolver(mip_gap=0.1 / 100, time_limit_seconds=60)

start = timeit.default_timer()

# Create resampled version for sizing
fs_sizing = flow_system.transform.resample('2h')
fs_sizing.optimize(solver)

timer_sizing = timeit.default_timer() - start
print(f'\nSizing optimization completed in {timer_sizing:.2f} seconds')
print(f'Timesteps: {len(fs_sizing.timesteps)}')

2025-12-08 15:15:32.682 DEBUG    │ Connected 4 Buses and 9 via 13 Flows inside the FlowSystem.
2025-12-08 15:15:32.683 DEBUG    │ Auto-registered carrier 'electricity' from CONFIG
2025-12-08 15:15:32.685 DEBUG    │ Auto-registered carrier 'heat' from CONFIG
2025-12-08 15:15:32.687 DEBUG    │ Auto-registered carrier 'gas' from CONFIG
2025-12-08 15:15:32.690 DEBUG    │ Auto-registered carrier 'fuel' from CONFIG
2025-12-08 15:15:32.693 DEBUG    │ Auto-assigned color '#636EFA' to component 'Kessel'
2025-12-08 15:15:32.699 DEBUG    │ Auto-assigned color '#EF553B' to component 'BHKW2'
2025-12-08 15:15:32.705 DEBUG    │ Auto-assigned color '#00CC96' to component 'Speicher'
2025-12-08 15:15:32.705 DEBUG    │ Auto-assigned color '#AB63FA' to component 'Wärmelast'
2025-12-08 15:15:32.706 DEBUG    │ Auto-assigned color '#FFA15A' to component 'Gastarif'
2025-12-08 15:15:32.709 DEBUG    │ Auto-assigned color '#19D3F3' to component 'Kohletarif'
2025-12-08 15:15:32.710 DEBUG    │ Auto-assigned color 

### Check Optimized Sizes

In [6]:
print('Optimized sizes from Stage 1:')
fs_sizing.statistics.sizes

Optimized sizes from Stage 1:


<xarray.Dataset> Size: 24B
Dimensions:       ()
Data variables:
    Kessel(Q_fu)  float64 8B 15.9
    BHKW2(Q_fu)   float64 8B 321.0
    Speicher      float64 8B 307.6

### Stage 2: Dispatch with Fixed Sizes

Now we use `transform.fix_sizes()` to create a new FlowSystem with sizes fixed from Stage 1, and optimize at full resolution:

In [7]:
start = timeit.default_timer()

# Fix sizes from Stage 1 and optimize at full resolution
fs_dispatch = flow_system.transform.fix_sizes(fs_sizing.statistics.sizes)
fs_dispatch.optimize(solver)

timer_dispatch = timeit.default_timer() - start
print(f'\nDispatch optimization completed in {timer_dispatch:.2f} seconds')
print(f'Timesteps: {len(fs_dispatch.timesteps)}')

2025-12-08 15:15:38.524 DEBUG    │ Skipping name='timesteps' because it is an Index
2025-12-08 15:15:38.551 INFO     │ Registered new Effect: costs
2025-12-08 15:15:38.552 INFO     │ Registered new Effect: CO2
2025-12-08 15:15:38.552 INFO     │ Registered new Effect: PE
2025-12-08 15:15:38.553 DEBUG    │ Fixed size of Kessel(Q_fu) to 15.89988
2025-12-08 15:15:38.555 DEBUG    │ Fixed size of BHKW2(Q_fu) to 321.00379
2025-12-08 15:15:38.556 DEBUG    │ Fixed size of Speicher to 307.58028
2025-12-08 15:15:38.556 DEBUG    │ Connected 4 Buses and 9 via 13 Flows inside the FlowSystem.
2025-12-08 15:15:38.557 DEBUG    │ Auto-assigned color '#636EFA' to component 'Kessel'
2025-12-08 15:15:38.557 DEBUG    │ Auto-assigned color '#EF553B' to component 'BHKW2'
2025-12-08 15:15:38.557 DEBUG    │ Auto-assigned color '#00CC96' to component 'Speicher'
2025-12-08 15:15:38.557 DEBUG    │ Auto-assigned color '#AB63FA' to component 'Wärmelast'
2025-12-08 15:15:38.558 DEBUG    │ Auto-assigned color '#FFA15A

Writing binary variables.: 100%|██████████| 12/12 [00:00<00:00, 768.99it/s]


Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
MIP linopy-problem-4ybq2ex4 has 29537 rows; 24539 cols; 88543 nonzeros; 6000 integer variables (6000 binary)
Coefficient ranges:
  Matrix  [1e-05, 1e+07]
  Cost    [1e+00, 1e+00]
  Bound   [1e+00, 1e+07]
  RHS     [1e-05, 1e+02]
Presolving model
16185 rows, 11883 cols, 42051 nonzeros  0s
13438 rows, 10759 cols, 39949 nonzeros  0s
Presolve reductions: rows 13438(-16099); columns 10759(-13780); nonzeros 39949(-48594) 

Solving MIP model with:
   13438 rows
   10759 cols (5376 binary, 0 integer, 2 implied int., 5381 continuous, 0 domain fixed)
   39949 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u =>

### Verify Size Consistency

In [8]:
sizing_sizes = fs_sizing.statistics.sizes
dispatch_sizes = fs_dispatch.statistics.sizes

print('Sizes comparison:')
for name in sizing_sizes.data_vars:
    sizing_val = float(sizing_sizes[name].item())
    dispatch_val = float(dispatch_sizes[name].item())
    print(f'  {name}: Sizing={sizing_val:.2f}, Dispatch={dispatch_val:.2f}')

Sizes comparison:
  Kessel(Q_fu): Sizing=15.90, Dispatch=15.90
  BHKW2(Q_fu): Sizing=321.00, Dispatch=321.00
  Speicher: Sizing=307.58, Dispatch=307.58


## Combined (Single-Stage) Optimization

For comparison, let's also solve the full problem in one go:

In [ ]:
start = timeit.default_timer()

# Full optimization with investment sizing
fs_combined = flow_system.copy()
fs_combined.optimize(fx.solvers.HighsSolver(mip_gap=0.1 / 100, time_limit_seconds=600))

timer_combined = timeit.default_timer() - start
print(f'\nCombined optimization completed in {timer_combined:.2f} seconds')

2025-12-08 15:15:48.641 DEBUG    │ Skipping name='timesteps' because it is an Index
2025-12-08 15:15:48.654 INFO     │ Registered new Effect: costs
2025-12-08 15:15:48.655 INFO     │ Registered new Effect: CO2
2025-12-08 15:15:48.655 INFO     │ Registered new Effect: PE
2025-12-08 15:15:48.656 DEBUG    │ Connected 4 Buses and 9 via 13 Flows inside the FlowSystem.
2025-12-08 15:15:48.656 DEBUG    │ Auto-assigned color '#636EFA' to component 'Kessel'
2025-12-08 15:15:48.657 DEBUG    │ Auto-assigned color '#EF553B' to component 'BHKW2'
2025-12-08 15:15:48.657 DEBUG    │ Auto-assigned color '#00CC96' to component 'Speicher'
2025-12-08 15:15:48.657 DEBUG    │ Auto-assigned color '#AB63FA' to component 'Wärmelast'
2025-12-08 15:15:48.658 DEBUG    │ Auto-assigned color '#FFA15A' to component 'Gastarif'
2025-12-08 15:15:48.658 DEBUG    │ Auto-assigned color '#19D3F3' to component 'Kohletarif'
2025-12-08 15:15:48.658 DEBUG    │ Auto-assigned color '#FF6692' to component 'Einspeisung'
2025-12-08

Writing binary variables.: 100%|██████████| 15/15 [00:00<00:00, 1243.37it/s]


Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
MIP linopy-problem-lcwu5bd7 has 29543 rows; 24542 cols; 88555 nonzeros; 6003 integer variables (6003 binary)
Coefficient ranges:
  Matrix  [1e-05, 1e+07]
  Cost    [1e+00, 1e+00]
  Bound   [1e+00, 1e+07]
  RHS     [1e-05, 2e+02]
Presolving model
17191 rows, 11889 cols, 45564 nonzeros  0s
15944 rows, 10765 cols, 49961 nonzeros  0s
Presolve reductions: rows 15944(-13599); columns 10765(-13777); nonzeros 49961(-38594) 

Solving MIP model with:
   15944 rows
   10765 cols (5379 binary, 0 integer, 2 implied int., 5384 continuous, 0 domain fixed)
   49961 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u =>

## Compare Results

### Key Metrics Comparison

In [ ]:
combined_sizes = fs_combined.statistics.sizes

comparison = pd.DataFrame(
    {
        'Combined': {
            'Duration [s]': timer_combined,
            'Total Costs [€]': fs_combined.solution['costs'].item(),
            'Kessel Size': combined_sizes['Kessel(Q_fu)'].item(),
            'BHKW2 Size': combined_sizes['BHKW2(Q_fu)'].item(),
            'Speicher Size': combined_sizes['Speicher'].item(),
        },
        'Two-Stage': {
            'Duration [s]': timer_sizing + timer_dispatch,
            'Total Costs [€]': fs_dispatch.solution['costs'].item(),
            'Kessel Size': dispatch_sizes['Kessel(Q_fu)'].item(),
            'BHKW2 Size': dispatch_sizes['BHKW2(Q_fu)'].item(),
            'Speicher Size': dispatch_sizes['Speicher'].item(),
        },
    }
).T

# Add difference row
combined_costs = comparison.loc['Combined', 'Total Costs [€]']
comparison['Cost Diff [%]'] = ((comparison['Total Costs [€]'] - combined_costs) / combined_costs * 100).round(2)

print(comparison.round(2))

### Visual Comparison

In [ ]:
# Compare heat balance
fs_combined.statistics.plot.balance('Fernwärme')

In [ ]:
fs_dispatch.statistics.plot.balance('Fernwärme')

## Analysis

The comparison shows:

| Metric | Combined | Two-Stage | Difference |
|--------|----------|-----------|------------|
| Duration | Longer | **Faster** | Significant speedup |
| Total Costs | Optimal | Near-optimal | Small suboptimality |
| Sizes | Optimal | Approximate | May differ slightly |

## When to Use Two-Stage Optimization

**Recommended for:**
- Large models (many timesteps, components)
- Development and debugging phases
- Screening studies where many configurations are tested
- Problems where sizing decisions are less sensitive to exact dispatch

**Not recommended for:**
- Small problems (overhead not worth it)
- Problems where sizing and dispatch are tightly coupled
- When exact optimal solutions are required

## Summary

This example demonstrated:

1. **`flow_system.transform.resample('2h')`** - Downsample time series for faster sizing
2. **`statistics.sizes`** - Extract optimized sizes from a solved FlowSystem
3. **`flow_system.transform.fix_sizes(sizes)`** - Create a new FlowSystem with fixed sizes
4. **Trade-off analysis** - Speed vs. optimality comparison

The two-stage approach provides a practical way to handle large-scale energy system models!